### Pulled from https://www.tensorflow.org/tutorials/keras/text_classification_with_hub

# Text classification with TensorFlow Hub: Movie reviews

This notebook classifies movie reviews as positive or negative using the text of the review. This is an example of binary—or two-class—classification, an important and widely applicable kind of machine learning problem.

The tutorial demonstrates the basic application of transfer learning with <a href="https://tfhub.dev/">TensorFlow Hub</a> and Keras.

It uses the <a href="https://www.tensorflow.org/api_docs/python/tf/keras/datasets/imdb">IMDB dataset</a> that contains the text of 50,000 movie reviews from the <a href="https://www.imdb.com/">Internet Movie Database.</a> These are split into 25,000 reviews for training and 25,000 reviews for testing. The training and testing sets are balanced, meaning they contain an equal number of positive and negative reviews.

This notebook uses <a href="https://www.tensorflow.org/guide/keras"><code>tf.keras</code></a>, a high-level API to build and train models in TensorFlow, and <a href="https://www.tensorflow.org/hub"><code>tensorflow_hub</code></a>, a library for loading trained models from <a href="https://tfhub.dev/">TFHub</a> in a single line of code. For a more advanced text classification tutorial using <a href="https://www.tensorflow.org/api_docs/python/tf/keras"><code>tf.keras</code></a>, see the <a href="https://developers.google.com/machine-learning/guides/text-classification/">MLCC Text Classification Guide.</a>

In [ ]:
!pip install tensorflow-hub
!pip install tensorflow-datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.1/644.9 MB 25.0 MB/s eta 0:00:20